In [1]:
%load_ext autoreload
%autoreload 2

In [27]:
from mnist_example import MNISTDataModule, MNISTModel

datamodule = MNISTDataModule(batch_size=10)
datamodule.setup()

len(datamodule.test_dataloader())

100

In [28]:
from energizer.data.datamodule import ActiveDataModule

dm = ActiveDataModule(datamodule=datamodule)
dm.setup()
dm.setup_folds()

dm.is_on_pool = False
print(len(dm.test_dataloader()))
dm.is_on_pool = True
print(len(dm.test_dataloader()))

100
100


In [29]:
# print(
#     len(dm.pool_fold),
#     len(dm.test_dataloader()),
#     len(dm.train_fold),
#     len(dm.train_dataloader()),
# )
# dm.label(list(range(100)))
# print(
#     len(dm.pool_fold),
#     len(dm.test_dataloader()),
#     len(dm.train_fold),
#     len(dm.train_dataloader()),
# )

---

In [30]:
import torch
from pytorch_lightning import LightningModule
from pytorch_lightning import Trainer as pl_Trainer
from torch.utils.data import DataLoader

from energizer.learners.acquisition_functions import entropy
from energizer.learners.base import Deterministic, Learner
from energizer.trainer import Trainer

In [31]:
class ActiveLearner(Deterministic):
    def pool_step(self, batch, *args, **kwargs) -> torch.Tensor:
        x, _ = batch
        logits = self(x)
        scores = entropy(logits)
        return scores

In [32]:
model = MNISTModel()
active_learner = ActiveLearner(model)

In [37]:
trainer = Trainer(
    query_size=2,
    max_epochs=5,
    max_labelling_iters=3,
    log_every_n_steps=1,
    test_after_labelling=True,
)
trainer.active_fit(active_learner, datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | learner | MNISTModel | 184 K 
---------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Epoch 0: 100%|██████████| 101/101 [00:00<00:00, 145.61it/s, loss=0.563, v_num=396]
1
1


HERE


Epoch 0: 100%|██████████| 102/102 [00:00<00:00, 172.45it/s, loss=0.549, v_num=396]
2
2


HERE


Epoch 0: 100%|██████████| 103/103 [00:00<00:00, 162.87it/s, loss=0.929, v_num=396]


In [36]:
trainer.num_val_batches, trainer.num_training_batches, trainer.num_test_batches

([100], 3, [98])

In [ ]:
pl_trainer = pl_Trainer(max_epochs=1)
pl_trainer.fit(active_learner, datamodule)

In [ ]:
pl_trainer.test(active_learner, datamodule)

In [ ]:
trainer.active_learning_loop.active_fit_loop.max_epochs

In [ ]:
test_dl = DataLoader(
    [datamodule.mnist_test[i] for i in range(10)],
    shuffle=False,
    batch_size=2,
)
x, y = next(iter(test_dl))

In [ ]:
active_learner(x).argmax(-1)

In [ ]:
y